In [1]:
## import libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.utils.data import Dataset, DataLoader

import os
import cv2 # for image imports
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print (f'Device is: {device}.')

Device is: cuda.


In [3]:
path = 'C:\\Users\\Toffee\\Documents\\Coding\\AI and ML\\DL from scratch\\data\\'

In [4]:
REBUILD_DATA = False # set to true to one once, then back to false unless you want to change something in your training data.

class DogsVSCats():
    IMG_SIZE = 64
    CATS = os.path.join(path, "PetImages\\Cat")
    DOGS = os.path.join(path, "PetImages\\Dog")
    TESTING = "PetImages\\Testing"
    LABELS = {CATS: 0, DOGS: 1}
    training_data = []

    catcount = 0
    dogcount = 0

    def make_training_data(self):
        for label in self.LABELS:
            print(label)
            for f in tqdm(os.listdir(label)):
                if "jpg" in f:
                    try:
                        path = os.path.join(label, f)
                        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
                        img = cv2.resize(img, (self.IMG_SIZE, self.IMG_SIZE))
                        self.training_data.append([np.array(img), np.eye(2)[self.LABELS[label]]])  # do something like print(np.eye(2)[1]), just makes one_hot 
                        #print(np.eye(2)[self.LABELS[label]])

                        if label == self.CATS:
                            self.catcount += 1
                        elif label == self.DOGS:
                            self.dogcount += 1

                    except Exception as e:
                        pass
                        #print(label, f, str(e))

        np.random.shuffle(self.training_data)
        np.save("training_data.npy", self.training_data)
        print('Cats:',dogsvcats.catcount)
        print('Dogs:',dogsvcats.dogcount)

In [5]:
class CNN(nn.Module):
    def __init__(self, input_size, n_features, output_size):
        super().__init__()
        self.n_features = n_features
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=n_features, kernel_size=5) # 64x64x1 -> 60x60xn_features -> pool
        self.conv2 = nn.Conv2d(n_features, n_features, kernel_size=5) # 30x30xn_features -> 26x26xn_features -> pool
        self.conv3 = nn.Conv2d(n_features, n_features, kernel_size=5) # 13x13xn_features -> 9x9xn_features -> pool
        self.fc1 = nn.Linear(n_features*4*4, 50)
        self.fc2 = nn.Linear(50, output_size)
        
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2)
        x = self.conv3(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2)
        x = x.view(-1, n_features*4*4)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.log_softmax(x, dim=1)
        return x

In [6]:
if REBUILD_DATA:
    dogsvcats = DogsVSCats()
    dogsvcats.make_training_data()

training_data = np.load("training_data.npy", allow_pickle=True)
print(len(training_data))

X = torch.Tensor([i[0] for i in training_data]).view(-1,64,64)
X = X/255.0
y = torch.Tensor([i[1] for i in training_data])

VAL_PCT = 0.1  # lets reserve 10% of our data for validation
val_size = int(len(X)*VAL_PCT)

train_X = X[:-val_size]
train_y = y[:-val_size]

test_X = X[-val_size:]
test_y = y[-val_size:]

24946


C:\Users\Toffee\AppData\Local\Temp\ipykernel_3228\1311000119.py:8: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:204.)
  X = torch.Tensor([i[0] for i in training_data]).view(-1,64,64)


In [7]:
input_size = 64*64
output_size = 2
n_features = 100

model = CNN(input_size, n_features, output_size).to(device)
optimizer = optim.Adam(model.parameters(),lr = 0.01)
criterion = nn.MSELoss()

In [8]:
BATCH_SIZE = 100
EPOCHS = 3

def train_cnn(model):
    model.train()
    for epoch in range(EPOCHS):
        for i in range(0, len(train_X), BATCH_SIZE): # from 0, to the len of x, stepping BATCH_SIZE at a time. [:50] ..for now just to dev
            #print(f"{i}:{i+BATCH_SIZE}")
            batch_X = train_X[i:i+BATCH_SIZE].view(-1, 1, 64, 64)
            batch_y = train_y[i:i+BATCH_SIZE]

            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            
            optimizer.zero_grad()   # zero the gradient buffers
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()    # Does the update

        if epoch % 10 == 0:
            print(f"Epoch: {epoch}. Loss: {loss}")

test_X.to(device)
test_y.to(device)

def test_cnn(model):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for i in tqdm(range(len(test_X))):
            real_class = torch.argmax(test_y[i]).to(device)
            model_out = model(test_X[i].view(-1, 1, 64, 64).to(device))[0]  # returns a list, 
            predicted_class = torch.argmax(model_out)

            if predicted_class == real_class:
                correct += 1
            total += 1

    print("Accuracy: ", round(correct/total*100, 3))

train_cnn(model)
test_cnn(model)

Epoch: 0. Loss: 1.6761013269424438


100%|██████████| 2494/2494 [00:02<00:00, 1154.49it/s]

Accuracy:  50.842
